# 03. Sleepscoring: brain-state classification

In this notebook, we will see an application of Fast Fourier Transform (FFT) and time-frequency analysis to a practical problem: differentiating brain states, with a specific focus on identifying sleep stages.

We will first load the data and then perform the analysis in two main steps:

- **EMG Analysis**: We will use the Electromyogram (EMG) to distinguish between wakefulness and sleep.
- **EEG Analysis**: We will then use the Electroencephalogram (EEG) to differentiate between Rapid Eye Movement (REM) and Non-REM (NREM) sleep.

Finally, we will combine these two analyses to construct a complete **hypnogram**, which is a graphical representation of the sleep stages over time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
# Load recording
rec = np.load("data/sleepscoring_traces.npz")
traces = rec["traces"] # Import traces
EEG = traces[0, :]  # EEG signal
EMG = traces[1, :]  # EMG signal
fs = rec["fs"]    # Sampling rate

In [ ]:
duration = traces.shape[1]/fs
t = np.arange(0, duration, 1/fs)

# I. EMG

## I.1 Computing the spectrogram

We start by computing the spectrogram of the EMG signal using the following parameters: 
-  window length of 4 seconds 
- overlap ratio of 50%

In [ ]:
window_length = 4.0 # seconds
overlapratio = 0.5  # 50% overlap
nperseg = int(window_length * fs)
noverlap = int(overlapratio * nperseg)
f, t_spec, Sxx = signal.spectrogram(EMG, fs, nperseg=nperseg, noverlap=noverlap)
Sxx_db = 10 * np.log10(Sxx + 1e-10)  # Convert to dB

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EMG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(-10,20)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

## I.2 Compute EMG Power

After computing the spectrogram, one straightforward method to distinguish between wake and sleep states is to calculate the total power across all frequencies. A higher power typically indicates wakefulness, while a lower power suggests sleep. This is just one of many possible approaches to perform this discrimination.

In [ ]:
# Sum power across frequencies (columns)
EMG_power = np.sum(Sxx_db, axis=0)

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 12))
gs = plt.GridSpec(3, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EMG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(-10,20)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax1 = fig.add_subplot(gs[2, 0])
ax1.plot(t_spec, EMG_power)
ax1.set_xlabel("Time(s)")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Power (dB)")
ax1.set_title("EMG Power")

## II.3 Setting threshold

Let's see the distribution of the EMG power

In [ ]:
plt.figure()
plt.hist(EMG_power, bins=50, density=True)
plt.xlabel('EMG Power')
plt.ylabel('Density')

We will now select a threshold based on the given data distribution to distinguish between wake and sleep states. For this example, we will manually select the threshold. However, in real-world applications, this process is typically automated using statistical methods (e.g., mean ± 2/3 standard deviation) or clustering algorithms to ensure accuracy and consistency.

In [ ]:
threshold_emg = -500

plt.figure()
plt.hist(EMG_power, bins=50, density=True)
plt.axvline(threshold_emg, color='green', linestyle='--', label='Threshold')
plt.xlabel('EMG Power')
plt.ylabel('Density')

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 12))
gs = plt.GridSpec(3, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EMG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(-10,20)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax1 = fig.add_subplot(gs[2, 0])
ax1.plot(t_spec, EMG_power)
ax1.axhline(threshold_emg, color='green', linestyle='--', label='Threshold')
ax1.set_xlabel("Time(s)")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Power (dB)")
ax1.set_title("EMG Power")

## II.4 Wake/Sleep classification

Based on this threshold we'll classify the signal in:
- WAKE if `EMG_power` > `threshold_emg`
- SLEEP if `EMG_power` < `threshold_emg`

In [ ]:
# Define sleep stage constants
UNCLASSIFIED = 1
WAKE = 0
REM = -1
NREM = -2
SLEEP = -3

In [ ]:
wakesleep_hypnogram = np.full(len(EMG_power), UNCLASSIFIED, dtype=int) # Create hypnogram array
wake_indices = (EMG_power > threshold_emg) # Identify wake indices
sleep_indices = ~wake_indices   # Identify sleep indices
wakesleep_hypnogram[wake_indices] = WAKE # Assign wake state
wakesleep_hypnogram[sleep_indices] = SLEEP  # Assign sleep state

In [ ]:
wakesleep_hypnogram

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 15))
gs = plt.GridSpec(4, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EMG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(-10,20)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax3 = fig.add_subplot(gs[2, 0])
ax3.plot(t_spec, EMG_power)
ax3.axhline(threshold_emg, color='green', linestyle='--', label='Threshold')
ax3.set_xlabel("")
ax3.set_xlim(0, duration)
ax3.set_ylabel("Power (dB)")
ax3.set_title("EMG Power")

# Plot WAKE/SLEEP classification
ax4 = fig.add_subplot(gs[3, 0])
ax4.plot(t_spec, wakesleep_hypnogram, 'k', drawstyle='steps-post')
ax4.set_xlabel("Time(s)")
ax4.set_xlim(0, duration)
ax4.set_yticks([SLEEP, WAKE], labels=["SLEEP", "WAKE"])
ax4.set_ylabel("Classification")
ax4.set_title("Hyonogram (WAKE/SLEEP)")

# II. EEG

## II.1 Computing the spectrogram

We now continue with the analysis of EEG brain signal, computing the spectrogram first

In [ ]:
window_length = 4.0 # seconds
overlapratio = 0.5  # 50% overlap
nperseg = int(window_length * fs)
noverlap = int(overlapratio * nperseg)
f, t_spec, Sxx = signal.spectrogram(EEG, fs, nperseg=nperseg, noverlap=noverlap)
Sxx_db = 10 * np.log10(Sxx + 1e-10)  # Convert to dB

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(0,30)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

From the EEG spectrogram, we can observe different brain states based on the varying frequencies captured. Some regions exhibit high power in the slow-wave bands (<4 Hz), while others lose this power and show increased activity in higher frequencies (around 6–7 Hz). Let's zoom in to analyze these transitions further.

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 8))
gs = plt.GridSpec(2, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(380, 470)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='gouraud', cmap="jet")
im.set_clim(0,30)
ax2.set_xlim(380, 470)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('Time (s)')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

## II.2 Computing REM/NREM metrics

To distinguish between NREM (slow-wave sleep) and REM sleep states, a common electrophysiological approach is to analyze the power spectral density of the EEG signal. Specifically, the ratio of power in different frequency bands is a robust metric.

One widely used metric is the theta/delta ratio. This is calculated by dividing the power in the theta band (typically 6-9 Hz) by the power in the delta band (typically 0.5-4 Hz).

- *High theta/delta ratio*: This indicates a predominance of theta activity over delta activity, which is a characteristic feature of REM sleep.
- *Low theta/delta ratio*: This indicates a predominance of delta activity, which is the hallmark of NREM sleep, particularly the deep, slow-wave stages.

Therefore, by setting a threshold on the theta/delta ratio, one can classify segments of sleep into either REM or NREM.

In [ ]:
delta = [0.5, 4]  # Hz
theta = [6, 9]  # Hz

delta_band = np.logical_and(f >= delta[0], f <= delta[1])   # Delta band indices
theta_band = np.logical_and(f >= theta[0], f <= theta[1])   # Theta band indices

delta_power = np.sum(Sxx_db[delta_band, :], axis=0)     # Sum delta power
theta_power = np.sum(Sxx_db[theta_band, :], axis=0)     # Sum theta power
theta_delta_ratio = theta_power / (delta_power + theta_power) # Compute theta/delta ratio

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 12))
gs = plt.GridSpec(3, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(0,30)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax3 = fig.add_subplot(gs[2, 0])
ax3.plot(t_spec, theta_delta_ratio)
ax3.set_xlabel("Time(s)")
ax3.set_xlim(0, duration)
ax3.set_ylabel("theta/delta ratio")
ax3.set_title("EMG Power")

As is evident from the plot, the raw theta/delta ratio calculated from real data exhibits significant oscillations. This high-frequency noise is an inherent characteristic of spectrogram-based analysis of biological signals.

To obtain a more stable and reliable metric for state classification, it is common practice to smooth the ratio time series. Smoothing helps to average out the rapid fluctuations and reveals the slower, underlying changes that correspond to transitions between sleep states. 

A common technique for this is applying a moving average filter.

In [ ]:
# Define smoothing window
window = 8  # epochs
moving_average_window = np.ones(window) / window

# Pad the signal at the borders
pad_width = window // 2
padded_signal = np.pad(theta_delta_ratio, pad_width, mode='edge')

# Apply convolution
smoothed_ratio = signal.convolve(padded_signal, moving_average_window, mode='same', method="fft")

# Remove the padding to restore the original length
smoothed_ratio = smoothed_ratio[pad_width:-pad_width]

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 12))
gs = plt.GridSpec(3, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(0,30)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax3 = fig.add_subplot(gs[2, 0])
ax3.plot(t_spec, smoothed_ratio)
ax3.set_xlabel("Time(s)")
ax3.set_xlim(0, duration)
ax3.set_ylabel("theta/delta ratio (smoothed)")
ax3.set_title("EMG Power")

## II.3 Setting threshold

In [ ]:
plt.figure()
plt.hist(smoothed_ratio, bins=50, density=True)
plt.xlabel('theta/delta ratio (smoothed)')
plt.ylabel('Density')

In [ ]:
threshold_eeg = 0.45

plt.figure()
plt.hist(smoothed_ratio, bins=50, density=True)
plt.axvline(threshold_eeg, color='green', linestyle='--', label='Threshold')
plt.xlabel('theta/delta ratio (smoothed)')
plt.ylabel('Density')

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 12))
gs = plt.GridSpec(3, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(0,30)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax3 = fig.add_subplot(gs[2, 0])
ax3.plot(t_spec, smoothed_ratio)
ax3.axhline(threshold_eeg, color='green', linestyle='--', label='Threshold')
ax3.set_xlabel("Time(s)")
ax3.set_xlim(0, duration)
ax3.set_ylabel("theta/delta ratio (smoothed)")
ax3.set_title("EMG Power")

## II.4 REM/NREM classification

Based on this threshold we'll classify the signal in:
- WAKE if `EMG_power` > `threshold_emg`
- REM if `not-WAKE` and `smoothed_ratio` > `threshold_eeg`
- SLEEP if `not-WAKE` and `smoothed_ratio` < `threshold_eeg`

In [ ]:
hypnogram = np.full(len(EMG_power), UNCLASSIFIED, dtype=int)
wake_indices = (EMG_power > threshold_emg)
sleep_indices = ~wake_indices
rem_indices = ((smoothed_ratio > threshold_eeg) & sleep_indices)
nrem_indices = ((smoothed_ratio <= threshold_eeg) & sleep_indices)


hypnogram[wake_indices] = WAKE
hypnogram[rem_indices] = REM
hypnogram[nrem_indices] = NREM

In [ ]:
hypnogram

In [ ]:
# Plot
fig = plt.figure(figsize=(17, 15))
gs = plt.GridSpec(4, 2, width_ratios=[15, .5], wspace=0.05)

# Plot the signal
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, EEG)
ax1.set_xlabel("")
ax1.set_xlim(0, duration)
ax1.set_ylabel("Voltage (uV)")
ax1.set_title("Signal")

# Plot the spectrogram
ax2 = fig.add_subplot(gs[1, 0])
im = ax2.pcolormesh(t_spec, f, Sxx_db, shading='auto', cmap="jet")
im.set_clim(0,30)
ax2.set_ylabel('Frequency [Hz]')
ax2.set_xlabel('')
ax2.set_title('Spectrogram')

# Colorbar
cax = fig.add_subplot(gs[1, 1])
fig.colorbar(im, cax=cax, label='Power')

# Plot the EMG power
ax3 = fig.add_subplot(gs[2, 0])
ax3.plot(t_spec, smoothed_ratio)
ax3.axhline(threshold_eeg, color='green', linestyle='--', label='Threshold')
ax3.set_xlabel("")
ax3.set_xlim(0, duration)
ax3.set_ylabel("Power (dB)")
ax3.set_title("EMG Power")

# Plot hypnogram
ax4 = fig.add_subplot(gs[3, 0])
ax4.plot(t_spec, hypnogram, 'k', drawstyle='steps-post')
ax4.set_xlabel("Time(s)")
ax4.set_xlim(0, duration)
ax4.set_yticks([NREM, REM, WAKE], labels=["NREM", "REM", "WAKE"])
ax4.set_ylabel("Classification")
ax4.set_title("Hyonogram")